In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

# Extract data from MIROC5-iso

This notebook allows to plot and extract data at a specific location and time range from MIROC5-iso simulations. Simulations nudged to ERA5 and JRA-55 are available. 
- The mean tritium in precipitation map can be plotted for a specified area and time period. The map can be downloaded as a pdf file.
- Time series of tritium and $\delta$$^{18}$O in precipitation, precipitation rate and air temperature at 2 m can be plotted for a specified location and time interval. The data can be downloaded as a CSV file, and the figure can be downloaded as a pdf file.
- The map of vertically integrated water vapor transport can be plotted for a given area and month. The figure can be downloaded as a pdf file. 

## Packages and path

In [2]:
import os
import wget
import numpy as np
import xarray as xr
import solara
import datetime
import ipywidgets as widgets
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
%matplotlib inline

In [3]:
# For the size of the figures
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 300
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')

In [4]:
# some shortcuts to download and read the miroc5-iso netcdf files
work ="data_tritium_bomb_miroc/"
exp_tritium = "bomb_tritium_"
exp_tritium_era5  = "bomb_tritium_era5"
exp_tritium_jra55 = "bomb_tritium_jra55"

## Download MIROC5-iso outputs

In [5]:
# download from isotope server
for var in ["prcpTU", "prcp", "temp2", "d18Op", "dexp", "evap", "qtot_u", "qtot_v"]:
    if os.path.isfile(exp_tritium_era5+"_miroc5_monmean."+var+".nc"):
        print(exp_tritium_era5+"_miroc5_monmean."+var+".nc was already downloaded")
        print(exp_tritium_jra55+"_miroc5_monmean."+var+".nc was already downloaded")
    else:
        print("")
        print("downloading "+exp_tritium_era5+"_miroc5_monmean."+var+".nc file")
        wget.download("http://isotope.iis.u-tokyo.ac.jp/~acauquoin/"+work+"/"+exp_tritium_era5+"_miroc5_monmean."+var+".nc")
        print("")
        print("downloading "+exp_tritium_jra55+"_miroc5_monmean."+var+".nc file")
        wget.download("http://isotope.iis.u-tokyo.ac.jp/~acauquoin/"+work+"/"+exp_tritium_jra55+"_miroc5_monmean."+var+".nc")       

bomb_tritium_era5_miroc5_monmean.prcpTU.nc was already downloaded
bomb_tritium_jra55_miroc5_monmean.prcpTU.nc was already downloaded
bomb_tritium_era5_miroc5_monmean.prcp.nc was already downloaded
bomb_tritium_jra55_miroc5_monmean.prcp.nc was already downloaded
bomb_tritium_era5_miroc5_monmean.temp2.nc was already downloaded
bomb_tritium_jra55_miroc5_monmean.temp2.nc was already downloaded
bomb_tritium_era5_miroc5_monmean.d18Op.nc was already downloaded
bomb_tritium_jra55_miroc5_monmean.d18Op.nc was already downloaded
bomb_tritium_era5_miroc5_monmean.dexp.nc was already downloaded
bomb_tritium_jra55_miroc5_monmean.dexp.nc was already downloaded
bomb_tritium_era5_miroc5_monmean.evap.nc was already downloaded
bomb_tritium_jra55_miroc5_monmean.evap.nc was already downloaded
bomb_tritium_era5_miroc5_monmean.qtot_u.nc was already downloaded
bomb_tritium_jra55_miroc5_monmean.qtot_u.nc was already downloaded
bomb_tritium_era5_miroc5_monmean.qtot_v.nc was already downloaded
bomb_tritium_jra55_

## Read and extract MIROC5-iso outputs

In [6]:
# read netcdf files
#
ifile_prcpTU_era5  = exp_tritium_era5  + "_miroc5_monmean.prcpTU.nc"
ifile_prcp_era5    = exp_tritium_era5  + "_miroc5_monmean.prcp.nc"
ifile_temp2_era5   = exp_tritium_era5  + "_miroc5_monmean.temp2.nc"
ifile_d18Op_era5   = exp_tritium_era5  + "_miroc5_monmean.d18Op.nc"
ifile_dexp_era5    = exp_tritium_era5  + "_miroc5_monmean.dexp.nc"
ifile_evap_era5    = exp_tritium_era5  + "_miroc5_monmean.evap.nc"
ifile_uqint_era5   = exp_tritium_era5  + "_miroc5_monmean.qtot_u.nc"
ifile_vqint_era5   = exp_tritium_era5  + "_miroc5_monmean.qtot_v.nc"
ifile_prcpTU_jra55 = exp_tritium_jra55 + "_miroc5_monmean.prcpTU.nc"
ifile_prcp_jra55   = exp_tritium_jra55 + "_miroc5_monmean.prcp.nc"
ifile_temp2_jra55  = exp_tritium_jra55 + "_miroc5_monmean.temp2.nc"
ifile_d18Op_jra55  = exp_tritium_jra55 + "_miroc5_monmean.d18Op.nc"
ifile_dexp_jra55   = exp_tritium_jra55 + "_miroc5_monmean.dexp.nc"
ifile_evap_jra55   = exp_tritium_jra55 + "_miroc5_monmean.evap.nc"
ifile_uqint_jra55  = exp_tritium_jra55 + "_miroc5_monmean.qtot_u.nc"
ifile_vqint_jra55  = exp_tritium_jra55 + "_miroc5_monmean.qtot_v.nc"
#
ds_prcpTU_era5  = xr.open_dataset(ifile_prcpTU_era5)
ds_prcp_era5    = xr.open_dataset(ifile_prcp_era5)
ds_temp2_era5   = xr.open_dataset(ifile_temp2_era5)
ds_d18Op_era5   = xr.open_dataset(ifile_d18Op_era5)
ds_dexp_era5    = xr.open_dataset(ifile_dexp_era5)
ds_evap_era5    = xr.open_dataset(ifile_evap_era5)
ds_uqint_era5   = xr.open_dataset(ifile_uqint_era5)
ds_vqint_era5   = xr.open_dataset(ifile_vqint_era5)
ds_prcpTU_jra55 = xr.open_dataset(ifile_prcpTU_jra55)
ds_prcp_jra55   = xr.open_dataset(ifile_prcp_jra55)
ds_temp2_jra55  = xr.open_dataset(ifile_temp2_jra55)
ds_d18Op_jra55  = xr.open_dataset(ifile_d18Op_jra55)
ds_dexp_jra55   = xr.open_dataset(ifile_dexp_jra55)
ds_evap_jra55   = xr.open_dataset(ifile_evap_jra55)
ds_uqint_jra55  = xr.open_dataset(ifile_uqint_jra55)
ds_vqint_jra55  = xr.open_dataset(ifile_vqint_jra55)
#
ds_prcpTU_era5  = ds_prcpTU_era5.isel(bnds=0)
ds_prcp_era5    = ds_prcp_era5.isel(bnds=0)
ds_temp2_era5   = ds_temp2_era5.isel(bnds=0)
ds_d18Op_era5   = ds_d18Op_era5.isel(bnds=0)
ds_dexp_era5    = ds_dexp_era5.isel(bnds=0)
ds_evap_era5    = ds_evap_era5.isel(bnds=0)
ds_uqint_era5   = xr.open_dataset(ifile_uqint_era5)
ds_uqint_era5   = ds_uqint_era5.isel(bnds=0)
ds_vqint_era5   = ds_vqint_era5.isel(bnds=0)
ds_prcpTU_jra55 = ds_prcpTU_jra55.isel(bnds=0)
ds_prcp_jra55   = ds_prcp_jra55.isel(bnds=0)
ds_temp2_jra55  = ds_temp2_jra55.isel(bnds=0)
ds_d18Op_jra55  = ds_d18Op_jra55.isel(bnds=0)
ds_dexp_jra55   = ds_dexp_jra55.isel(bnds=0)
ds_evap_jra55   = ds_evap_jra55.isel(bnds=0)
ds_uqint_jra55  = ds_uqint_jra55.isel(bnds=0)
ds_vqint_jra55  = ds_vqint_jra55.isel(bnds=0)
#
ds_prcp_era5   = ds_prcp_era5.rename({"pr": "precip"})
ds_temp2_era5  = ds_temp2_era5.rename({"tas": "temp2"})
ds_evap_era5   = ds_evap_era5.rename({"hfls": "evap"})
ds_prcp_jra55  = ds_prcp_jra55.rename({"pr": "precip"})
ds_temp2_jra55 = ds_temp2_jra55.rename({"tas": "temp2"})
ds_evap_jra55  = ds_evap_jra55.rename({"hfls": "evap"})
#
ds_prcpTU_era5.TU_precip.attrs['units'] = "TU"
ds_prcpTU_era5.TU_precip.attrs['standard_name'] = "tritium in precipitation"
ds_prcp_era5.precip.attrs['units'] = "mm/month"
ds_prcp_era5.precip.attrs['standard_name'] = "precipitation"
ds_temp2_era5.temp2.attrs['units'] = "°C"
ds_temp2_era5.temp2.attrs['standard_name'] = "2m air temperature"
ds_d18Op_era5.d18O_precip.attrs['units'] = "‰"
ds_d18Op_era5.d18O_precip.attrs['standard_name'] = "d18O in precipitation"
ds_dexp_era5.dex_precip.attrs['units'] = "‰"
ds_dexp_era5.dex_precip.attrs['standard_name'] = "d-excess in precipitation"
ds_evap_era5.evap.attrs['standard_name'] = "surface upward latent heat flux"
ds_evap_era5.evap.attrs['units'] = "W m-2"
ds_uqint_era5.UQ.attrs['units'] = "kg m-1 s-1"
ds_uqint_era5.UQ.attrs['standard_name'] = "vert. int. water vapor transport - u component"
ds_vqint_era5.VQ.attrs['units'] = "kg m-1 s-1"
ds_vqint_era5.VQ.attrs['standard_name'] = "vert. int. water vapor transport - v component"
ds_prcpTU_jra55.TU_precip.attrs['units'] = "TU"
ds_prcpTU_jra55.TU_precip.attrs['standard_name'] = "tritium in precipitation"
ds_prcp_jra55.precip.attrs['units']      = "mm/month"
ds_prcp_jra55.precip.attrs['standard_name'] = "precipitation"
ds_temp2_jra55.temp2.attrs['units']      = "°C"
ds_temp2_jra55.temp2.attrs['standard_name'] = "2m air temperature"
ds_d18Op_jra55.d18O_precip.attrs['units'] = "‰"
ds_d18Op_jra55.d18O_precip.attrs['standard_name'] = "d18O in precipitation"
ds_dexp_jra55.dex_precip.attrs['units'] = "‰"
ds_dexp_jra55.dex_precip.attrs['standard_name'] = "d-excess in precipitation"
ds_evap_jra55.evap.attrs['standard_name'] = "surface upward latent heat flux"
ds_evap_jra55.evap.attrs['units'] = "W m-2"
ds_uqint_jra55.UQ.attrs['units'] = "kg m-1 s-1"
ds_uqint_jra55.UQ.attrs['standard_name'] = "vert. int. water vapor transport - u component"
ds_vqint_jra55.VQ.attrs['units'] = "kg m-1 s-1"
ds_vqint_jra55.VQ.attrs['standard_name'] = "vert. int. water vapor transport - v component"
#
variables = ['TU_precip', 'precip', 'temp2', 'd18O_precip', 'dex_precip', 'evap', 'UQ', 'VQ']
ds_era5  = xr.merge([ds_prcpTU_era5,  ds_prcp_era5,  ds_temp2_era5, ds_d18Op_era5, ds_dexp_era5, ds_evap_era5, ds_uqint_era5, ds_vqint_era5])
ds_era5 = ds_era5[variables]
ds_jra55 = xr.merge([ds_prcpTU_jra55, ds_prcp_jra55, ds_temp2_jra55, ds_d18Op_jra55, ds_dexp_jra55, ds_evap_jra55, ds_uqint_jra55, ds_vqint_jra55])
ds_jra55 = ds_jra55[variables]

In [7]:
def convert360_180(_ds):
    """
    convert longitude from 0-360 to -180 -- 180 deg
    """
    # check if already 
    attrs = _ds['lon'].attrs
    if _ds['lon'].min() >= 0:
        with xr.set_options(keep_attrs=True): 
            _ds.coords['lon'] = (_ds['lon'] + 180) % 360 - 180
        _ds = _ds.sortby('lon')
    return _ds

In [8]:
ds_era5  = convert360_180(ds_era5)
ds_jra55 = convert360_180(ds_jra55)

In [9]:
ds_era5

<xarray.Dataset>
Dimensions:      (time: 840, lat: 64, lon: 128)
Coordinates:
  * time         (time) datetime64[ns] 1952-01-16T12:00:00 ... 2021-12-16T12:...
  * lon          (lon) float64 -180.0 -177.2 -174.4 -171.6 ... 171.6 174.4 177.2
  * lat          (lat) float64 87.86 85.1 82.31 79.53 ... -82.31 -85.1 -87.86
Data variables:
    TU_precip    (time, lat, lon) float32 ...
    precip       (time, lat, lon) float32 ...
    temp2        (time, lat, lon) float32 ...
    d18O_precip  (time, lat, lon) float32 ...
    dex_precip   (time, lat, lon) float32 ...
    evap         (time, lat, lon) float32 ...
    UQ           (time, lat, lon) float32 ...
    VQ           (time, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 2.0.4 (https://mpimet.mpg.de...
    Conventions:  CF-1.0
    history:      Thu Sep 07 17:47:18 2023: cdo -f nc -setreftime,1952-01-01 ...
    CDO:          Climate Data Operators version 2.0.4 (https://mpimet.mpg.de...

## Map of tritium in precipitation

In [10]:
def map_tritium(ndg, time_min, time_max, lon_min, lon_max, lat_min, lat_max):
    
    global f
    
    # ERA5 or JRA55
    if ndg == 'era5':
        ds = ds_era5
    elif ndg == 'jra55':
        ds = ds_jra55
    
    # conversion of some variables
    time_min = time_min.strftime("%Y-%m")
    time_max = time_max.strftime("%Y-%m")
    lon_min = float(lon_min)
    lon_max = float(lon_max)
    lat_min = float(lat_min)
    lat_max = float(lat_max)
    if time_min == time_max:
        title = 'Mean values for ' + ds.sel(time=time_min).time.dt.strftime('%Y-%m').values[0]
    else:
        title = 'Mean values over the period ' + ds.sel(time=time_min).time.dt.strftime('%Y-%m').values[0] + ' to ' + ds.sel(time=time_max).time.dt.strftime('%Y-%m').values[0]
    
    f,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
    
    TU_weight = ds['TU_precip'] * ds['precip']
    num = TU_weight.sel(time=slice(time_min, time_max), lon=slice(lon_min, lon_max), lat=slice(lat_max, lat_min)).sum(axis=0)
    denom = ds['precip'].sel(time=slice(time_min, time_max), lon=slice(lon_min, lon_max), lat=slice(lat_max, lat_min)).sum(axis=0)
    TU_mean = num / denom
    
    TU_mean.plot.pcolormesh(
        cmap = plt.get_cmap('RdYlBu_r'),ax=ax,
        cbar_kwargs={
            'label':'Tritium in precipitation (TU)', 
            'orientation':'horizontal',
            'pad':0.03, 
            'aspect':30
        }, 
        transform=ccrs.PlateCarree(), 
        x='lon',y='lat', levels=30, 
        edgecolors='face', shading='nearest')
    
    ax.coastlines()
    ax.set_title(title)
    
def save_image_tritium(b):
    f.savefig('tritium_map.pdf', bbox_inches='tight')

In [11]:
def build_tritium_map():
    
    # create widget to select the simulation nudged to ERA5 or JRA-55
    ndg = widgets.Dropdown(options=[('ERA5', 'era5'), ('JRA-55', 'jra55')], value='era5', 
                           description='nudging:', 
                           continuous_update=False,
                           layout=widgets.Layout(width='200px'))
    
    # create widget for coordinates selection
    lon_min = widgets.Text(value="-180", 
                           placeholder='between -180 and 180', 
                           description='lon min:', 
                           disabled=False, 
                           continuous_update=False,
                           layout=widgets.Layout(width='200px'))
    lon_max = widgets.Text(value="180", 
                           placeholder='between -180 and 180', 
                           description='lon max:', 
                           disabled=False, 
                           continuous_update=False,
                           layout=widgets.Layout(width='200px'))
    lat_min = widgets.Text(value="-90", 
                           placeholder='between -90 and 90', 
                           description='lat min:', 
                           disabled=False, 
                           continuous_update=False,
                           layout=widgets.Layout(width='200px'))
    lat_max = widgets.Text(value="90", 
                           placeholder='between -90 and 90', 
                           description='lat max:', 
                           disabled=False, 
                           continuous_update=False,
                           layout=widgets.Layout(width='200px'))
    coord_lon = lon_min, lon_max
    coord_lat = lat_min, lat_max
    
    # create widget for the date range
    time_min = widgets.DatePicker(description='date min:', disabled=False, continuous_update=False, value = datetime.date(2021,1,1))
    time_max = widgets.DatePicker(description='date max:', disabled=False, continuous_update=False, value = datetime.date(2021,12,1))
    time_input = time_min, time_max
    
    # Arrange the control inputs:
    controls = widgets.HBox([
        ndg,
        widgets.VBox(coord_lon),
        widgets.VBox(coord_lat),
        widgets.VBox(time_input)
    ])
    
    # link the map_tritium function with the text input
    plot = widgets.interactive_output(map_tritium, 
                                      {'ndg': ndg, 
                                       'time_min': time_min,
                                       'time_max': time_max,
                                       'lon_min': lon_min,
                                       'lon_max': lon_max,
                                       'lat_min': lat_min,
                                       'lat_max': lat_max
                                      })
    
    # button to save the map
    btn_save = widgets.Button(description="save figure", icon='file-pdf', layout=widgets.Layout(margin='1em 0 0 0'))
    btn_save.on_click(save_image_tritium)

    # Build the UI panel, the controls at the top, the cdo output at the bottom
    comment1 = widgets.HTML('Note 1: the day in the date pickers is not considered.', layout=widgets.Layout(margin='1em 1em 0 0'))
    comment2 = widgets.HTML('Note 2: The average value of each grid cell for the period between \'date min\' and \'date max\' is calculated.', layout=widgets.Layout(margin='0 1em 0 0'))
    comment3 = widgets.HTML('Note 3: the colorbar scale is updated according to the mean tritium values in the considered area.', layout=widgets.Layout(margin='0 1em 0 0'))
    comment4 = widgets.HTML('Note 4: for the coordinate boxes, type \'Enter\' or click on another box to update your change.', layout=widgets.Layout(margin='0 1em 0 0'))
    comment5 = widgets.HTML('Note 5: the figure is saved in the current repository as tritium_map.pdf.', layout=widgets.Layout(margin='0 1em 0 0'))
    ui = widgets.VBox([plot, controls, btn_save, comment1, comment2, comment3, comment4, comment5])
    
    return ui

In [12]:
build_tritium_map()

## Plot MIROC5-iso data at selected location and time

In [13]:
def plot_and_save_data_site(lon_station, lat_station, 
                            ndg, var, date_min, date_max, 
                            logscale, 
                            save_tu, save_precip, save_evap, save_temp2, save_d18Op, save_dexp):
    
    global fig
    
    # nudging choice
    if ndg == 'era5':
        ds = ds_era5
    elif ndg == 'jra55':
        ds = ds_jra55
    save_var = [save_tu, save_precip, save_evap, save_temp2, save_d18Op, save_dexp]
    
    # log scale
    if logscale == True:
        yscale = 'log'
    else:
        yscale = 'linear'
    
    # conversion of some variables
    date_min = date_min.strftime("%Y-%m")
    date_max = date_max.strftime("%Y-%m")
    lon_station = float(lon_station)
    lat_station = float(lat_station)
    
    # selection of date
    ds_sel_date = ds.sel(time=slice(date_min, date_max))
    
    # plotting the data on the nearest grid cell of the chosen location
    fig, ax = plt.subplots(figsize=(8,3), constrained_layout=True)
    ax = ds_sel_date[var].sel(lon=lon_station, lat=lat_station, 
                              method='nearest').plot(x='time', 
                                                     ax=ax, yscale=yscale)
    
    # download button
    cols = []
    var_plot = ['TU_precip', 'precip', 'evap', 'temp2', 'd18O_precip', 'dex_precip']
    for i in range(len(var_plot)):
        if save_var[i] == True:
            cols.append(var_plot[i])
    ds_sel_date = ds_sel_date.sel(lon=lon_station, lat=lat_station, method='nearest')
    df = ds_sel_date.to_dataframe()
    df = df[cols]
    df.index = df.index.strftime('%Y-%m')
    w = widgets.VBox([solara.FileDownload.widget(data=df.to_csv(sep=';'), filename="data_miroc5-iso.csv", layout=widgets.Layout(margin='1em 0 0 0'))])
    display(w)
    
    # print message
    print("")
    print("Information of the plot:")
    print("Variable:", var)
    print("Longitude:", lon_station, "; Latitude:", lat_station)
    print("Nudging:", ndg)
    print("Date range:", date_min, "to", date_max)

    
def save_monitoring_plot(b):
    fig.savefig('monitoring_plot.pdf', bbox_inches='tight')

In [14]:
# update the tritium time series based on coordinates values
def build_monitoring_plot():
    
    # create widget for coordinates selection
    lon_station = widgets.Text(value="0", 
                               placeholder='between -180 and 180', description='Longitude:', 
                               disabled=False, continuous_update=False, layout=widgets.Layout(width='250px'))
    lat_station = widgets.Text(value="0", 
                               placeholder='between -90 and 90', description='Latitude:', 
                               disabled=False, continuous_update=False, layout=widgets.Layout(width='250px'))
    coord_input = lon_station, lat_station
    
    # create widget to select the simulation nudged to ERA5 or JRA-55
    ndg = widgets.Dropdown(options=[('ERA5', 'era5'), ('JRA-55', 'jra55')], value='era5', 
                         description='Nudging:', continuous_update=False, layout=widgets.Layout(width='275px'))
    
    # create widget to select the variable
    selvar = widgets.Dropdown(options=[('TU in precipitation', 'TU_precip'), 
                                       ('2m air temperature', 'temp2'), 
                                       ('precipitation', 'precip'),
                                       ('evapotranspiration', 'evap'),
                                       ('d18O in precipitation', 'd18O_precip'),
                                       ('d-excess in precipitation', 'dex_precip')], 
                              value='TU_precip', description='Variable:',
                              continuous_update=False,
                              layout=widgets.Layout(width='275px'))
    
    # create a widget to set the y-scale on log or linear scale
    check_logscale = widgets.Checkbox(value=False, description="log scale", continuous_update=False,
                                      indent=False, layout=widgets.Layout(margin='0 0 0 2.5em'))
    
    # create widget to select the variables to save in the csv file
    data = ["TU in precipitation", "Precipitation", "Evapotranspiration", "2m air temperature", "d18O in precipitation", "d-excess in precipitation"]
    checkboxes = [widgets.Checkbox(value=False, description=label, indent=False, continuous_update=False,
                                   layout=widgets.Layout(width='175px', margin='0 0 0 0')) for label in data]
    
    # create widget for the date range
    date_min = widgets.DatePicker(description='Date min:', disabled=False, continuous_update=False, value = datetime.date(1952,1,1))
    date_max = widgets.DatePicker(description='Date max:', disabled=False, continuous_update=False, value = datetime.date(2021,12,1))
    date_input = date_min, date_max
    
    # button to save the time series
    btn_save = widgets.Button(description="save figure", icon='file-pdf')
    btn_save.on_click(save_monitoring_plot)
    
    # Arrange the control inputs:
    controls = widgets.HBox([
        widgets.VBox(coord_input),
        widgets.VBox(date_input),
        widgets.VBox([ndg, selvar]),
        check_logscale,
        btn_save
    ])
    
    # link the data extraction command with the text input
    plot = widgets.interactive_output(plot_and_save_data_site, 
                                      {'lon_station': lon_station, 'lat_station': lat_station,
                                       'ndg': ndg, 'var': selvar,
                                       'date_min': date_min, 'date_max': date_max,
                                       'logscale': check_logscale,
                                       'save_tu': checkboxes[0], 
                                       'save_precip': checkboxes[1],
                                       'save_evap': checkboxes[2],
                                       'save_temp2': checkboxes[3],
                                       'save_d18Op': checkboxes[4],
                                       'save_dexp': checkboxes[5]})
    
    # Build the UI panel, the controls at the top, the cdo output at the bottom
    label = widgets.HTML('<b>Select the variables to download:</b>', layout=widgets.Layout(margin='0 1em 0 0'))
    comment1 = widgets.HTML('Note 1: the day in the date pickers is not considered.', layout=widgets.Layout(margin='1em 1em 0 0'))
    comment2 = widgets.HTML('Note 2: for the coordinate boxes, type \'Enter\' or click on another box to update your change.', layout=widgets.Layout(margin='0 1em 0 0'))
    comment3 = widgets.HTML('Note 3: the evapotranspiration is expressed as latent heat flux in W m<sup>-2</sup>.', layout=widgets.Layout(margin='0 1em 0 0'))
    comment4 = widgets.HTML('Note 4: the figure is saved in the current repository as monitoring_plot.pdf.', layout=widgets.Layout(margin='0 1em 0 0'))
    ui = widgets.VBox([widgets.HBox([label, widgets.HBox(children=checkboxes)]), plot, controls, comment1, comment2, comment3, comment4])
    
    return ui

In [15]:
build_monitoring_plot()

## Map of water vapor transport

In [16]:
def map_water_vapor_transport(ndg, date, lon_min, lon_max, lat_min, lat_max, spacing):
    
    global f_wt
    
    # ERA5 or JRA55
    if ndg == 'era5':
        ds = ds_era5
    elif ndg == 'jra55':
        ds = ds_jra55
    
    # conversion of some variables
    date    = date.strftime("%Y-%m")
    lon_min = float(lon_min)
    lon_max = float(lon_max)
    lat_min = float(lat_min)
    lat_max = float(lat_max)
    spacing = int(spacing)
    
    title = 'Vertically integrated water vapor transport for ' + ds.sel(time=date).time.dt.strftime('%Y-%m').values[0]
    
    # Select month, coordinates, and calculate wind speed
    ds_quiver = ds[['UQ', 'VQ']].sel(time=date, lon=slice(lon_min, lon_max, spacing), lat=slice(lat_max, lat_min, spacing)).squeeze()
    ds_u = ds['UQ'].sel(time=date, lon=slice(lon_min, lon_max), lat=slice(lat_max, lat_min)).squeeze()
    ds_v = ds['VQ'].sel(time=date, lon=slice(lon_min, lon_max), lat=slice(lat_max, lat_min)).squeeze()
    ds_speed = np.sqrt(ds_v**2 + ds_u**2)
    
    # figure
    f_wt,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
    
    ds_speed.plot.pcolormesh(
        cmap = plt.get_cmap('BrBG'), ax=ax,
        cbar_kwargs={
            'label':'Vertically integrated water vapor transport (kg.m$^{-1}$.s$^{-1}$)', 
            'orientation':'horizontal',
            'pad':0.03, 
            'aspect':30
        },
        transform=ccrs.PlateCarree(), 
        x='lon',y='lat', levels=30, 
        edgecolors='face', shading='nearest')
    
    ax.coastlines()
    ax.set_title(title)
    Q = ax.quiver(ds_quiver['lon'], ds_quiver['lat'], ds_quiver['UQ'], ds_quiver['VQ'], scale=None, transform=ccrs.PlateCarree())
    Q._init()
    assert isinstance(Q.scale, float)
    ax.quiver(ds_quiver['lon'], ds_quiver['lat'], ds_quiver['UQ'], ds_quiver['VQ'], scale_units='inches', scale=Q.scale, transform=ccrs.PlateCarree())
    
def save_image_transport(b):
    f_wt.savefig('water_vapor_transport_map.pdf', bbox_inches='tight')

In [17]:
def build_water_vapor_trans_map():
    
    # create widget to select the simulation nudged to ERA5 or JRA-55
    ndg = widgets.Dropdown(options=[('ERA5', 'era5'), ('JRA-55', 'jra55')], value='era5', 
                           description='nudging:', 
                           continuous_update=False,
                           layout=widgets.Layout(width='250px'))
    
    # create widget for coordinates selection
    lon_min = widgets.Text(value="-180", 
                           placeholder='between -180 and 180', 
                           description='lon min:', 
                           disabled=False, 
                           continuous_update=False,
                           layout=widgets.Layout(width='200px'))
    lon_max = widgets.Text(value="180", 
                           placeholder='between -180 and 180', 
                           description='lon max:', 
                           disabled=False, 
                           continuous_update=False,
                           layout=widgets.Layout(width='200px'))
    lat_min = widgets.Text(value="-90", 
                           placeholder='between -90 and 90', 
                           description='lat min:', 
                           disabled=False, 
                           continuous_update=False,
                           layout=widgets.Layout(width='200px'))
    lat_max = widgets.Text(value="90", 
                           placeholder='between -90 and 90', 
                           description='lat max:', 
                           disabled=False, 
                           continuous_update=False,
                           layout=widgets.Layout(width='200px'))
    coord_lon = lon_min, lon_max
    coord_lat = lat_min, lat_max
    
    # create widget for coordinates spacing (for the vectors)
    spacing = widgets.Text(value="1", 
                           placeholder='higher or equal to 1', 
                           description='vector spacing:', 
                           disabled=False, 
                           continuous_update=False,
                           layout=widgets.Layout(width='250px'),
                           style= {'description_width': 'initial'})
    
    # create widget for the date selection
    date = widgets.DatePicker(description='date:', disabled=False, continuous_update=False, value = datetime.date(2021,1,1))
    
    # Arrange the control inputs:
    controls = widgets.HBox([
        widgets.VBox([ndg, spacing]),
        date,
        widgets.VBox(coord_lon),
        widgets.VBox(coord_lat)
    ])
    
    # link the map_tritium function with the text input
    plot = widgets.interactive_output(map_water_vapor_transport, 
                                      {'ndg': ndg, 
                                       'date': date,
                                       'lon_min': lon_min,
                                       'lon_max': lon_max,
                                       'lat_min': lat_min,
                                       'lat_max': lat_max,
                                       'spacing': spacing
                                      })
    
    # button to save the map
    btn_save = widgets.Button(description="save figure", icon='file-pdf', layout=widgets.Layout(margin='1em 0 0 0'))
    btn_save.on_click(save_image_transport)

    # Build the UI panel, the controls at the top, the cdo output at the bottom
    comment1 = widgets.HTML('Note 1: the day in the date pickers is not considered.', layout=widgets.Layout(margin='1em 1em 0 0'))
    comment2 = widgets.HTML('Note 2: the colorbar scale is updated according to the mean tritium values in the considered area.', layout=widgets.Layout(margin='0 1em 0 0'))
    comment3 = widgets.HTML('Note 3: for the coordinate boxes, type \'Enter\' or click on another box to update your change.', layout=widgets.Layout(margin='0 1em 0 0'))
    comment4 = widgets.HTML('Note 4: spacing parameter controls the space between vectors (1: every grid cell, 2: every two grid cells, ...).', layout=widgets.Layout(margin='0 1em 0 0'))
    comment5 = widgets.HTML('Note 5: the figure is saved in the current repository as water_vapor_transport_map.pdf.', layout=widgets.Layout(margin='0 1em 0 0'))
    ui = widgets.VBox([plot, controls, btn_save, comment1, comment2, comment3, comment4, comment5])
    
    return ui

In [18]:
build_water_vapor_trans_map()

## Packages version

In [19]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p wget,numpy,xarray,solara,ipywidgets,matplotlib,cartopy,watermark

# date
print (" ")
%watermark -u -n -t -z

Python implementation: CPython
Python version       : 3.11.2
IPython version      : 8.12.0

wget      : 3.2
numpy     : 1.24.2
xarray    : 2023.3.0
solara    : 1.19.0
ipywidgets: 8.0.6
matplotlib: 3.6.3
cartopy   : 0.21.1
watermark : 2.4.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 22.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 10
Architecture: 64bit

 
Last updated: Tue Oct 24 2023 14:17:30JST

